In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -q /content/drive/My\ Drive/upper_zip.zip

In [0]:
!rm -r /content/upper/1

In [0]:
from glob import glob
import numpy as np

for i in range(0,4):
  to_test_dir = "/content/upper_new/"+str(i)+"/*"
  arr = np.array(glob(to_test_dir))
  print(arr.shape)

(4308,)
(4308,)
(4308,)
(4308,)


In [0]:
to_test_dir = "/content/upper_temp/*"
arr = np.array(glob(to_test_dir))
print(arr.shape)

(6012,)


In [0]:
!mkdir upper_temp

In [0]:
import shutil

min_num = 4308
to_move = np.random.permutation(np.array(glob("/content/upper_temp/*")))
for file in to_move[:min_num]:
  shutil.move(file,"/content/upper_new/0")

In [0]:
!zip -r -q upper_new_zip upper_new/ *.jpg

In [0]:
!mv upper_new_zip.zip /content/drive/My\ Drive/

In [0]:
!pip install split-folders

In [0]:
import split_folders
split_folders.ratio("/content/upper_new",output="/content/upper_data",ratio=(.7,.1,.2))

In [0]:
import torch
torch.cuda.current_device()

0

In [0]:
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import transforms
import torchvision.models as models
import numpy as np
from tqdm import tqdm
from PIL import Image
from PIL import ImageFile
use_cuda = torch.cuda.is_available()

In [0]:
img_std = [0.229,0.224,0.225]
img_mean = [0.485,0.456,0.406]
batch_size = 32

train_transforms = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(img_mean,img_std)
                                     ])
train_dir = "/content/upper_data/train"
valid_dir = "/content/upper_data/val"
test_dir = "/content/upper_data/test"

train_dataset = datasets.ImageFolder(train_dir,transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir,transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir,transform=train_transforms)

trainloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
validloader = torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

loaders = {
    'train':trainloader,
    'valid':validloader,
    'test':testloader
}

In [0]:
class UpperModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder_net = models.resnet101(pretrained=True)
    self.encoder_net = nn.Sequential(*list(self.encoder_net.children())[:-1])
    
    self.fc1 = nn.Sequential(nn.Dropout(0.45),
                            nn.Linear(2048,256),
                            nn.BatchNorm1d(num_features=256),
                            nn.ReLU(True),
                            nn.Dropout(0.2),
                            nn.Linear(256,4),
                            nn.LogSoftmax(dim=1))
    
  def forward(self,x):
    encoding = self.encoder_net(x).view((-1,2048))
    preds = self.fc1(encoding)

    return preds
    
model = UpperModel()
if use_cuda:
  model.cuda()

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth
100%|██████████| 178728960/178728960 [00:18<00:00, 9833730.06it/s] 


In [0]:
import torch.optim as optim

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.parameters(),lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.2)

In [0]:
def train(n_epochs,model,criterion,optimizer,scheduler,loaders,use_cuda,save_path):
  valid_loss_min = np.Inf
  valid_acc_min = 0
  
  for epoch in range(1,n_epochs+1):
    valid_loss = 0
    train_loss = 0
    valid_corrects = 0
    train_corrects = 0
    
    scheduler.step()
    
    # Training loop
    model.train()
    for data,target in tqdm(loaders['train']):
      if use_cuda:
        data,target = data.cuda(),target.cuda()
        
      optimizer.zero_grad()
      
      output = model(data)
      
      loss = criterion(output,target)
      
      loss.backward()
      
      optimizer.step()
      
      train_loss += loss.item()
      
      pred = output.data.max(1,keepdim=True)[1]
      train_corrects += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
      
    
    #Validation loop
    model.eval()
    for data,target in tqdm(loaders['valid']):
      if use_cuda:
        data,target = data.cuda(),target.cuda()
       
      with torch.no_grad():
        output = model(data)
      
      loss = criterion(output,target)
      
      valid_loss += loss.item()
      
      pred = output.data.max(1,keepdim=True)[1]
      valid_corrects += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
    
    valid_loss = valid_loss / len(loaders['valid'])
    train_loss = train_loss / len(loaders['train'])
    valid_accuracy = valid_corrects / (len(loaders['valid'])*batch_size)
    train_accuracy = train_corrects / (len(loaders['train'])*batch_size)
      
    #Print epoch details
    print("Epoch: {}, Training loss: {}, Training accuracy: {}, Valid loss: {}, Valid accuracy; {}".format(epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))
    
    if valid_loss < valid_loss_min:
      valid_loss_min = valid_loss
      valid_acc_min = valid_accuracy
      checkpoint = model.state_dict()
      torch.save(checkpoint,save_path)
      
  print("The best model had a validation loss of {} and an accuracy of {}".format(valid_loss_min,valid_acc_min))

In [0]:
model = train(15,model,criterion,optimizer,scheduler,loaders,use_cuda,"/content/drive/My Drive/upper_5.pt")

100%|██████████| 54/54 [00:23<00:00,  2.44it/s]


Epoch: 1, Training loss: 0.7882696392523515, Training accuracy: 0.7106266578249337, Valid loss: 0.9446697003311582, Valid accuracy; 0.6446759259259259


100%|██████████| 54/54 [00:22<00:00,  2.44it/s]


Epoch: 2, Training loss: 0.6374319532505713, Training accuracy: 0.7700596816976127, Valid loss: 0.6765265393036383, Valid accuracy; 0.7528935185185185


100%|██████████| 54/54 [00:22<00:00,  2.44it/s]


...

Epoch: 12, Training loss: 0.07401622215221668, Training accuracy: 0.9823441644562334, Valid loss: 0.4876468843883938, Valid accuracy; 0.8587962962962963


  0%|          | 0/377 [00:00<?, ?it/s]

Epoch: 13, Training loss: 0.07373604034160113, Training accuracy: 0.9828415119363395, Valid loss: 0.4866600828590216, Valid accuracy; 0.8541666666666666


 72%|███████▏  | 273/377 [05:55<02:15,  1.31s/it]

KeyboardInterrupt: ignored